# Save CSVs

In [ ]:
import sys
import importlib
import numpy as np
import nbformat
import plotly.express
import plotly.express as px
import pandas as pd
import scipy.optimize as optimization
import food_bank_functions
import food_bank_bayesian
from food_bank_functions import *
from food_bank_bayesian import *
importlib.reload(food_bank_functions)

In [ ]:
opt_policy = np.loadtxt('opt_policy.csv', delimiter=",")

# Experiments

## Varying Number of Groups

In [ ]:
num_iterations = 1000
max_n = 1000
grid_size = .01
sorted_distribution = np.arange(2) + 1
weights = np.zeros(2)+0.5
expected_demand = np.dot(sorted_distribution,weights)
max_budget = max_n*expected_demand
b_grid = np.arange(0, max_budget+grid_size, grid_size)

### L1 Norm between OPT and ALGs

In [ ]:
data_dict_1 = {'NumGroups':[],'Dynamic':[],'Weights':[], 'Bayes':[], 'Threshold':[],'Greedy':[]}
for n in np.logspace(0,3,100):
    n = max(int(n),1)
    group_expected_demands = np.zeros(n) + expected_demand    
    budget = n*expected_demand
    for i in range(num_iterations):
        data_dict_1['NumGroups'].append(n)
        group_demands = np.random.randint(1, 3, n)
        opt = waterfilling_waste(group_demands,budget)
        data_dict_1['Weights'].append(np.sum(np.absolute(opt - waterfilling_weights_waste(weights, sorted_distribution, group_demands, budget))))
        data_dict_1['Dynamic'].append(np.sum(np.absolute(opt - waterfilling_dynamic_waste(group_expected_demands,group_demands,budget))))
        data_dict_1['Bayes'].append(np.sum(np.absolute(opt - waterfilling_bayesian(group_demands, opt_policy[(max_n-n):(max_n+1),:], budget, b_grid, grid_size))))
        data_dict_1['Greedy'].append(np.sum(np.absolute(opt - greedy(group_demands,budget))))
        data_dict_1['Threshold'].append(np.sum(np.absolute(opt - constant_threshold(group_demands,budget,expected_demand))))                                                  
df_uniform = pd.DataFrame(data_dict_1).melt(id_vars="NumGroups")
df_uniform.to_csv('L1.csv')
fig = px.scatter(df_uniform, x="NumGroups", y="value", color='variable')
fig.update_layout(xaxis_type="log", yaxis_type="log")
fig.update_layout(
    title="L1 Distance between OPT and ALGs",
    xaxis_title="Number of Groups",
    yaxis_title="Distance")
fig.show()

### L-Inf Norm between OPT and ALGs

In [ ]:
data_dict_1 = {'NumGroups':[],'Dynamic':[],'Weights':[], 'Bayes':[], 'Threshold':[],'Greedy':[]}

for n in np.logspace(0,3,100):
    n = max(int(n),1)
    group_expected_demands = np.zeros(n) + expected_demand    
    budget = n*expected_demand
    for i in range(num_iterations):
        data_dict_1['NumGroups'].append(n)
        group_demands = np.random.randint(1, 3, n)
        opt = waterfilling_waste(group_demands,budget)
        data_dict_1['Weights'].append(np.amax(np.absolute(opt - waterfilling_weights_waste(weights, sorted_distribution, group_demands, budget))))
        data_dict_1['Dynamic'].append(np.amax(np.absolute(opt - waterfilling_dynamic_waste(group_expected_demands,group_demands,budget))))
        data_dict_1['Bayes'].append(np.amax(np.absolute(opt - waterfilling_bayesian(group_demands, opt_policy[(max_n-n):(max_n+1),:], budget, b_grid, grid_size))))
        data_dict_1['Greedy'].append(np.amax(np.absolute(opt - greedy(group_demands,budget))))
        data_dict_1['Threshold'].append(np.amax(np.absolute(opt - constant_threshold(group_demands,budget,expected_demand))))
df_uniform = pd.DataFrame(data_dict_1).melt(id_vars="NumGroups")
df_uniform.to_csv('L-Inf.csv')
fig = px.scatter(df_uniform, x="NumGroups", y="value", color='variable')
fig.update_layout(xaxis_type="log", yaxis_type="log")
fig.update_layout(
    title="L-Inf Distance between OPT and ALGs",
    xaxis_title="Number of Groups",
    yaxis_title="Distance")
fig.show()

### Nash Welfare Regret wrt OPT

In [ ]:
data_dict_1 = {'NumGroups':[],'Dynamic':[],'Weights':[], 'Bayes':[], 'Threshold':[],'Greedy':[]}

for n in np.logspace(0,3,100):
    n = max(int(n),1)
    group_expected_demands = np.zeros(n) + expected_demand    
    budget = n*expected_demand
    for i in range(num_iterations):
        data_dict_1['NumGroups'].append(n)
        group_demands = np.random.randint(1, 3, n)
        opt = objective_nash(group_demands, waterfilling_waste(group_demands,budget))
        data_dict_1['Weights'].append(opt - objective_nash(group_demands, waterfilling_weights_waste(weights, sorted_distribution, group_demands, budget)))
        data_dict_1['Dynamic'].append(opt - objective_nash(group_demands, waterfilling_dynamic_waste(group_expected_demands,group_demands,budget)))
        data_dict_1['Bayes'].append(opt - objective_nash(group_demands, waterfilling_bayesian(group_demands, opt_policy[(max_n-n):(max_n+1),:], budget, b_grid, grid_size)))
        data_dict_1['Threshold'].append(opt - objective_nash(group_demands, constant_threshold(group_demands,budget,expected_demand)))
        data_dict_1['Greedy'].append(opt - objective_nash(group_demands, greedy(group_demands,budget)))
df_uniform = pd.DataFrame(data_dict_1).melt(id_vars="NumGroups")
df_uniform.to_csv('Nash.csv')
fig = px.scatter(df_uniform, x="NumGroups", y="value", color='variable')
fig.update_layout(xaxis_type="log", yaxis_type="log")
fig.update_layout(
    title="Nash Welfare Regret wrt OPT",
    xaxis_title="Number of Groups",
    yaxis_title="Regret")
fig.show()

### Log Nash Welfare Regret wrt OPT

In [ ]:
data_dict_1 = {'NumGroups':[],'Dynamic':[],'Weights':[], 'Bayes':[], 'Threshold':[],'Greedy':[]}

for n in np.logspace(0,3,100):
    n = max(int(n),1)
    group_expected_demands = np.zeros(n) + expected_demand    
    budget = n*expected_demand
    for i in range(num_iterations):
        data_dict_1['NumGroups'].append(n)
        group_demands = np.random.randint(1, 3, n)
        opt = objective_nash_log(group_demands, waterfilling_waste(group_demands,budget))
        data_dict_1['Weights'].append(opt - objective_nash_log(group_demands, waterfilling_weights_waste(weights, sorted_distribution, group_demands, budget)))
        data_dict_1['Dynamic'].append(opt - objective_nash_log(group_demands, waterfilling_dynamic_waste(group_expected_demands,group_demands,budget)))
        data_dict_1['Bayes'].append(opt - objective_nash_log(group_demands, waterfilling_bayesian(group_demands, opt_policy[(max_n-n):(max_n+1),:], budget, b_grid, grid_size)))
        data_dict_1['Greedy'].append(opt - objective_nash_log(group_demands, greedy(group_demands,budget)))
        data_dict_1['Threshold'].append(opt - objective_nash_log(group_demands, constant_threshold(group_demands,budget,expected_demand)))
df_uniform = pd.DataFrame(data_dict_1).melt(id_vars="NumGroups")
df_uniform.to_csv('LogNash.csv')
fig = px.scatter(df_uniform, x="NumGroups", y="value", color='variable')
fig.update_layout(xaxis_type="log", yaxis_type="log")
fig.update_layout(
    title="Log Nash Welfare Regret  wrt OPT",
    xaxis_title="Number of Groups",
    yaxis_title="Regret")
fig.show()

### NORMALIZED Log Nash Welfare Regret wrt OPT

In [ ]:
data_dict_1 = {'NumGroups':[],'Dynamic':[],'Weights':[], 'Bayes':[], 'Threshold':[],'Greedy':[]}

for n in np.logspace(0,3,100):
    n = max(int(n),1)
    group_expected_demands = np.zeros(n) + expected_demand    
    budget = n*expected_demand
    for i in range(num_iterations):
        data_dict_1['NumGroups'].append(n)
        group_demands = np.random.randint(1, 3, n)
        opt = objective_nash_log_normalized(group_demands, waterfilling_waste(group_demands,budget))
        data_dict_1['Weights'].append(opt - objective_nash_log_normalized(group_demands, waterfilling_weights_waste(weights, sorted_distribution, group_demands, budget)))
        data_dict_1['Dynamic'].append(opt - objective_nash_log_normalized(group_demands, waterfilling_dynamic_waste(group_expected_demands,group_demands,budget)))
        data_dict_1['Bayes'].append(opt - objective_nash_log_normalized(group_demands, waterfilling_bayesian(group_demands, opt_policy[(max_n-n):(max_n+1),:], budget, b_grid, grid_size)))
        data_dict_1['Greedy'].append(opt - objective_nash_log_normalized(group_demands, greedy(group_demands,budget)))
        data_dict_1['Threshold'].append(opt - objective_nash_log_normalized(group_demands, constant_threshold(group_demands,budget,expected_demand)))
df_uniform = pd.DataFrame(data_dict_1).melt(id_vars="NumGroups")
df_uniform.to_csv('LogNashNormal.csv')
fig = px.scatter(df_uniform, x="NumGroups", y="value", color='variable')
fig.update_layout(xaxis_type="log", yaxis_type="log")
fig.update_layout(
    title="Normalized Log Nash Welfare Regret wrt OPT",
    xaxis_title="Number of Groups",
    yaxis_title="Regret")
fig.show()

## Constant Number of Groups: Group by Group Comparison

In [ ]:
num_iterations = 1000
grid_size = .01
sorted_distribution = np.arange(2) + 1
weights = np.zeros(2)+0.5
expected_demand = np.dot(sorted_distribution,weights)
n=200

## Fairness Comparisons using Sum of Values

In [ ]:
group = np.arange(n)
group_expected_demands = np.zeros(n) + 1.5
budget = 1.5*n
score_weights = np.zeros((n,num_iterations))
score_bayes = np.zeros((n,num_iterations))
score_dynamic = np.zeros((n,num_iterations))
score_greedy = np.zeros((n,num_iterations))
score_threshold = np.zeros((n,num_iterations))
env = np.zeros((5,num_iterations))
po = np.zeros((5,num_iterations))
prop = np.zeros((5,num_iterations))
for i in range(num_iterations):
    group_demands = np.random.randint(1, 3, n)
    opt = waterfilling_waste(group_demands,budget)
    bayes = waterfilling_bayesian(group_demands, opt_policy[(max_n-n):(max_n+1),:], budget, b_grid, grid_size)
    score_bayes[:,i] = opt - bayes
    env[0,i] = max(0,np.sum(envy_vector(bayes,group_demands)))
    po[0,i] = excess(bayes,budget)
    prop[0,i] = np.sum(proportionality(bayes,group_demands,budget))
    weights_alloc = waterfilling_weights_waste(weights, sorted_distribution, group_demands,budget)
    score_weights[:,i] = opt - weights_alloc
    env[1,i] = max(0,np.sum(envy_vector(weights_alloc,group_demands)))
    po[1,i] = excess(weights_alloc,budget)
    prop[1,i] = np.sum(proportionality(weights_alloc,group_demands,budget))
    dynamic = waterfilling_dynamic_waste(group_expected_demands,group_demands,budget)
    score_dynamic[:,i] = opt - dynamic
    env[2,i] = max(0,np.sum(envy_vector(dynamic,group_demands)))
    po[2,i] = excess(dynamic,budget)
    prop[2,i] = np.sum(proportionality(dynamic,group_demands,budget))
    greedy_alloc = greedy(group_demands,budget)
    score_greedy[:,i] = opt - greedy_alloc
    env[3,i] = max(0,np.sum(envy_vector(greedy_alloc,group_demands)))
    po[3,i] = excess(greedy_alloc,budget)
    prop[3,i] = np.sum(proportionality(greedy_alloc,group_demands,budget))
    threshold = constant_threshold(group_demands,budget,expected_demand)
    score_threshold[:,i] = opt - threshold
    env[4,i] = max(0,np.sum(envy_vector(threshold,group_demands)))
    po[4,i] = excess(threshold,budget)
    prop[4,i] = np.sum(proportionality(threshold,group_demands,budget))
score_bayes = np.average(score_bayes, axis=1)
score_weights = np.average(score_weights, axis=1)
score_dynamic = np.average(score_dynamic, axis=1)
score_greedy = np.average(score_greedy, axis=1)
score_threshold = np.average(score_threshold, axis=1)
env = np.average(env,axis=1)
po = np.average(po,axis=1)
prop = np.average(prop,axis=1)
print('bayes, weights, dynamic, greedy, threshold')
print('envy:')
print(env)
print('po')
print(po)
print('prop')
print(prop)
data_dict = {'Group':group,'Dynamic':score_dynamic,'Weights':score_weights, 'Bayes':score_bayes, 'Threshold':score_threshold,'Greedy':score_greedy}
df_uniform = pd.DataFrame(data_dict).melt(id_vars="Group")
df_uniform.to_csv('GroupAllocationDifference.csv')
fig = px.scatter(df_uniform, x="Group", y="value", color='variable')
fig.update_layout(
    title="Group by Group Allocation Difference between OPT and ALGs",
    xaxis_title="Group",
    yaxis_title="Difference")
fig.show()
#[0.119     0.0288263 0.        0.666     0.       ]

## Fairness Comparisons using Max Value

In [ ]:
group = np.arange(n)
group_expected_demands = np.zeros(n) + 1.5
budget = 1.5*n
score_weights = np.zeros((n,num_iterations))
score_bayes = np.zeros((n,num_iterations))
score_dynamic = np.zeros((n,num_iterations))
score_greedy = np.zeros((n,num_iterations))
score_threshold = np.zeros((n,num_iterations))
env = np.zeros((5,num_iterations))
po = np.zeros((5,num_iterations))
prop = np.zeros((5,num_iterations))
for i in range(num_iterations):
    group_demands = np.random.randint(1, 3, n)
    opt = waterfilling_waste(group_demands,budget)
    bayes = waterfilling_bayesian(group_demands, opt_policy[(max_n-n):(max_n+1),:], budget, b_grid, grid_size)
    score_bayes[:,i] = opt - bayes
    env[0,i] = max(0,np.amax(envy_vector(bayes,group_demands)))
    po[0,i] = excess(bayes,budget)
    prop[0,i] = np.amax(proportionality(bayes,group_demands,budget))
    weights_alloc = waterfilling_weights_waste(weights, sorted_distribution, group_demands,budget)
    score_weights[:,i] = opt - weights_alloc
    env[1,i] = max(0,np.amax(envy_vector(weights_alloc,group_demands)))
    po[1,i] = excess(weights_alloc,budget)
    prop[1,i] = np.amax(proportionality(weights_alloc,group_demands,budget))
    dynamic = waterfilling_dynamic_waste(group_expected_demands,group_demands,budget)
    score_dynamic[:,i] = opt - dynamic
    env[2,i] = max(0,np.amax(envy_vector(dynamic,group_demands)))
    po[2,i] = excess(dynamic,budget)
    prop[2,i] = np.amax(proportionality(dynamic,group_demands,budget))
    greedy_alloc = greedy(group_demands,budget)
    score_greedy[:,i] = opt - greedy_alloc
    env[3,i] = max(0,np.amax(envy_vector(greedy_alloc,group_demands)))
    po[3,i] = excess(greedy_alloc,budget)
    prop[3,i] = np.amax(proportionality(greedy_alloc,group_demands,budget))
    threshold = constant_threshold(group_demands,budget,expected_demand)
    score_threshold[:,i] = opt - threshold
    env[4,i] = max(0,np.amax(envy_vector(threshold,group_demands)))
    po[4,i] = excess(threshold,budget)
    prop[4,i] = np.amax(proportionality(threshold,group_demands,budget))
score_bayes = np.average(score_bayes, axis=1)
score_weights = np.average(score_weights, axis=1)
score_dynamic = np.average(score_dynamic, axis=1)
score_greedy = np.average(score_greedy, axis=1)
score_threshold = np.average(score_threshold, axis=1)
env = np.average(env,axis=1)
po = np.average(po,axis=1)
prop = np.average(prop,axis=1)
print('bayes, weights, dynamic, greedy, threshold')
print('envy:')
print(env)
print('po')
print(po)
print('prop')
print(prop)
data_dict = {'Group':group,'Dynamic':score_dynamic,'Weights':score_weights, 'Bayes':score_bayes, 'Threshold':score_threshold,'Greedy':score_greedy}
df_uniform = pd.DataFrame(data_dict).melt(id_vars="Group")
df_uniform.to_csv('GroupAllocationDifference.csv')
fig = px.scatter(df_uniform, x="Group", y="value", color='variable')
fig.update_layout(
    title="Group by Group Allocation Difference between OPT and ALGs",
    xaxis_title="Group",
    yaxis_title="Difference")
fig.show()
#[0.119     0.0288263 0.        0.666     0.       ]

## Fairness Comparisons using Utility

In [ ]:
group = np.arange(n)
group_expected_demands = np.zeros(n) + 1.5
budget = 1.5*n
score_weights = np.zeros((n,num_iterations))
score_bayes = np.zeros((n,num_iterations))
score_dynamic = np.zeros((n,num_iterations))
score_greedy = np.zeros((n,num_iterations))
score_threshold = np.zeros((n,num_iterations))
env = np.zeros((5,num_iterations))
po = np.zeros((5,num_iterations))
prop = np.zeros((5,num_iterations))
linf = np.zeros((5,num_iterations))
for i in range(num_iterations):
    group_demands = np.random.randint(1, 3, n)
    opt = waterfilling_waste(group_demands,budget)
    bayes = waterfilling_bayesian(group_demands, opt_policy[(max_n-n):(max_n+1),:], budget, b_grid, grid_size)
    score_bayes[:,i] = opt - bayes
    env[0,i] = max(0,np.amax(envy_utility(bayes,group_demands)))
    po[0,i] = excess(bayes,budget)
    prop[0,i] = np.amax(proportionality_utility(bayes,group_demands,budget))
    linf[0,i] = np.amax(score_bayes[:,i])
    weights_alloc = waterfilling_weights_waste(weights, sorted_distribution, group_demands,budget)
    score_weights[:,i] = opt - weights_alloc
    env[1,i] = max(0,np.amax(envy_utility(weights_alloc,group_demands)))
    po[1,i] = excess(weights_alloc,budget)
    prop[1,i] = np.amax(proportionality_utility(weights_alloc,group_demands,budget))
    linf[1,i] = np.amax(score_weights[:,i])
    dynamic = waterfilling_dynamic_waste(group_expected_demands,group_demands,budget)
    score_dynamic[:,i] = opt - dynamic
    env[2,i] = max(0,np.amax(envy_utility(dynamic,group_demands)))
    po[2,i] = excess(dynamic,budget)
    prop[2,i] = np.amax(proportionality_utility(dynamic,group_demands,budget))
    linf[2,i] = np.amax(score_dynamic[:,i])
    greedy_alloc = greedy(group_demands,budget)
    score_greedy[:,i] = opt - greedy_alloc
    env[3,i] = max(0,np.amax(envy_utility(greedy_alloc,group_demands)))
    po[3,i] = excess(greedy_alloc,budget)
    prop[3,i] = np.amax(proportionality_utility(greedy_alloc,group_demands,budget))
    linf[3,i] = np.amax(score_greedy[:,i])
    threshold = constant_threshold(group_demands,budget,expected_demand)
    score_threshold[:,i] = opt - threshold
    env[4,i] = max(0,np.amax(envy_utility(threshold,group_demands)))
    po[4,i] = excess(threshold,budget)
    prop[4,i] = np.amax(proportionality_utility(threshold,group_demands,budget))
    linf[4,i] = np.amax(score_threshold[:,i])
score_bayes = np.average(score_bayes, axis=1)
score_weights = np.average(score_weights, axis=1)
score_dynamic = np.average(score_dynamic, axis=1)
score_greedy = np.average(score_greedy, axis=1)
score_threshold = np.average(score_threshold, axis=1)
env = np.average(env,axis=1)
po = np.average(po,axis=1)
prop = np.average(prop,axis=1)
linf = np.average(linf,axis=1)
print('bayes, weights, dynamic, greedy, threshold')
print('envy:')
print(env)
print('po')
print(po)
print('prop')
print(prop)
print('linf')
print(linf)
data_dict = {'Group':group,'Dynamic':score_dynamic,'Weights':score_weights, 'Bayes':score_bayes, 'Threshold':score_threshold,'Greedy':score_greedy}
df_uniform = pd.DataFrame(data_dict).melt(id_vars="Group")
#df_uniform.to_csv('GroupAllocationDifference.csv')
fig = px.scatter(df_uniform, x="Group", y="value", color='variable')
fig.update_layout(
    title="Group by Group Allocation Difference between OPT and ALGs",
    xaxis_title="Group",
    yaxis_title="Difference")
fig.show()